In [55]:
import pickle
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from datetime import date
from collections import namedtuple

In [56]:
import cafle as cf
from cafle.genfunc import rounding as R
from cafle.genfunc import percent as P

In [57]:
pd.options.display.max_columns = 40
pd.options.display.max_rows = 200
print(pd.get_option("display.max_columns"),
      pd.get_option("display.max_rows"))

40 200


# Initial Setting

In [59]:
def ctg(nml, cld):
    idx = ['nml', 'cld']
    return Series([nml, cld], index = idx)

In [60]:
# Pickling Data
with open("data/area.pickle", "rb") as fr:
    data_area = pickle.load(fr)
areadf = data_area['areadf']

In [61]:
# Call out the area data
area_rent = areadf.loc[[x in data_area['rent_col'] for x in areadf.name]]
area_rent = area_rent.area_py.groupby(areadf['floor']).sum()
area_cld = round(area_rent[['b1', 'f1']].sum(), 2)
area_nml = round(area_rent[['f2', 'f3', 'f4']].sum(), 2)

### Valuation Class

In [62]:
class Valuation:
    def __init__(self, rent, mtnc, cap_rate, area, vcncy_rate, oprtg_rate, IR, RU=1_000_000):
        self.rent = rent
        self.mtnc = mtnc
        self.cap_rate = cap_rate
        self.area = area
        self.vcncy_rate = vcncy_rate
        self.oprtg_rate = oprtg_rate
        self.IR = IR
        self.RU = RU
        
        self.__intlz__()
        self.ctgr = ['rent', 'mtnc', 'cap_rate', 'area', 'vcncy_rate', 'oprtg_rate',
                     'PGI', 'EGI', 'oprtg_cst', 'NOI', 'value']
        self.ctgr_rate = ['cap_rate', 'vcncy_rate', 'oprtg_rate']
        

    def __intlz__(self):
        self.dpst_amt = self.rent * 10 * self.area / self.RU # deposit amount
        self.rent_amt = self.rent * 12 * self.area / self.RU # yearly rent amount
        self.mtnc_amt = self.mtnc * 12 * self.area / self.RU # yearly maintenance cost
        
        self.PGI = (self.dpst_amt * self.IR) + self.rent_amt + self.mtnc_amt
        self.EGI = self.PGI * (1 - self.vcncy_rate)
        self.oprtg_cst = self.PGI * self.oprtg_rate
        self.NOI = self.EGI - self.oprtg_cst
        self.value = self.NOI / self.cap_rate
        self.value_sum = self.value.sum()
        
    def printval(self, withsum=True):
        tmpdct = DataFrame({x: getattr(self, x) for x in self.ctgr})
        if withsum:
            tmpdct_withsum = tmpdct.append(DataFrame([tmpdct.sum()], index=['sum']))
            return DataFrame(tmpdct_withsum)
        else:
            return DataFrame(tmpdct)

In [63]:
class Val_Mtrx:
    def __init__(self, rent=None, mtnc=None, cap_rate=None, area=None, 
                 vcncy_rate=None, oprtg_rate=None, IR=None, RU=1_000_000):
        self.rent = rent
        self.mtnc = mtnc
        self.cap_rate=cap_rate
        self.area = area
        self.vcncy_rate = vcncy_rate
        self.oprtg_rate = oprtg_rate
        self.IR = IR
        self.RU = RU
        
        self.ctgr = ['rent', 'mtnc', 'cap_rate', 'area', 'vcncy_rate', 'oprtg_rate',
                     'PGI', 'EGI', 'oprtg_cst', 'NOI', 'value']
        self.ctgr_rate = ['cap_rate', 'vcncy_rate', 'oprtg_rate']
        self.ctgr_cld = ['nml', 'cld']
        
        self.rslt_lst = []
    
    
    @property
    def ctgr_rate_mlti(self):
        ctgr_rslt = []
        for val1 in self.ctgr_rate:
            for val2 in self.ctgr_cld:
                ctgr_rslt.append((val1, val2))
        return ctgr_rslt
    
        
    def value(self, rent=None, mtnc=None, cap_rate=None, area=None, 
                 vcncy_rate=None, oprtg_rate=None, IR=None):
        keys = ['rent', 'mtnc', 'cap_rate', 'area', 'vcncy_rate', 'oprtg_rate', 'IR']
        dct = {}
        for key in keys:
            if locals()[key] is not None:
                tmp_val = locals()[key]
            else:
                tmp_val = getattr(self, key)
            dct['tmp_'+key] = tmp_val
            
        tmpistnc = Valuation(rent = dct['tmp_rent'],
                             mtnc = dct['tmp_mtnc'],
                             cap_rate = dct['tmp_cap_rate'],
                             area = dct['tmp_area'],
                             vcncy_rate = dct['tmp_vcncy_rate'],
                             oprtg_rate = dct['tmp_oprtg_rate'],
                             IR = dct['tmp_IR'])
        return tmpistnc

                
    def mkdict(self, **kwargs):
        lenarg = len(kwargs)
        keylst = []
        arglst = []
        for key, item in kwargs.items():
            keylst.append(key)
            arglst.append(item)
        
        for val0 in arglst[0]:
            if lenarg == 1:
                tmpdct = {keylst[0]:val0}
                self.rslt_lst.append(self.value(**tmpdct))
            else:
                for val1 in arglst[1]:
                    if lenarg == 2:
                        tmpdct = {keylst[0]:val0, keylst[1]:val1}
                        self.rslt_lst.append(self.value(**tmpdct))
                    else:
                        for val2 in arglst[2]:
                            if lenarg == 3:
                                tmpdct = {keylst[0]:val0, keylst[1]:val1, keylst[2]:val2}
                                self.rslt_lst.append(self.value(**tmpdct))

    
    def printlst(self):
        tmplst = []
        tmpsum = []
        for rslt in self.rslt_lst:
            tmprslt = rslt.printval(withsum=False).unstack()
            tmplst.append(tmprslt)
            
            tmpsum.append(rslt.value_sum)
        tmpdf = DataFrame(tmplst)
        tmpdf[('value', 'sum')] = tmpsum
        return tmpdf

# Valuation
### 1) Assumption

In [64]:
val_mtrx = Val_Mtrx(rent = ctg(20_000, 44_000),
                    cap_rate = ctg(P(4.3), P(4.8)),
                    mtnc = ctg(2_000, 2_000),
                    area = ctg(area_nml, area_cld),
                    vcncy_rate = ctg(0.0, 0.0),
                    oprtg_rate = P(14.5),
                    IR = P(1.0))

In [65]:
rent_case = [ctg(20_000, 44_000), # unit rent
             ctg(21_000, 46_000),
             ctg(22_000, 48_000),
             ctg(23_000, 50_000)]
cap_rate = [ctg(P(4.3), P(4.8)),
            ctg(P(4.4), P(4.9)),
            ctg(P(4.5), P(5.0))]

val_mtrx.mkdict(rent=rent_case, cap_rate=cap_rate)
val_mtrx.rslt_lst

### 2) Pickling Result

In [66]:
data_valuation = {}

In [67]:
valuation_mtrx = val_mtrx.printlst()
data_valuation['valuation_mtrx'] = valuation_mtrx
R(valuation_mtrx, rate=val_mtrx.ctgr_rate_mlti)

rent           mtnc        cap_rate        area        vcncy_rate       \
       nml     cld    nml    cld      nml  cld    nml    cld        nml  cld   
0   20,000  44,000  2,000  2,000      4.3  4.8  8,082  6,619        0.0  0.0   
1   20,000  44,000  2,000  2,000      4.4  4.9  8,082  6,619        0.0  0.0   
2   20,000  44,000  2,000  2,000      4.5  5.0  8,082  6,619        0.0  0.0   
3   21,000  46,000  2,000  2,000      4.3  4.8  8,082  6,619        0.0  0.0   
4   21,000  46,000  2,000  2,000      4.4  4.9  8,082  6,619        0.0  0.0   
5   21,000  46,000  2,000  2,000      4.5  5.0  8,082  6,619        0.0  0.0   
6   22,000  48,000  2,000  2,000      4.3  4.8  8,082  6,619        0.0  0.0   
7   22,000  48,000  2,000  2,000      4.4  4.9  8,082  6,619        0.0  0.0   
8   22,000  48,000  2,000  2,000      4.5  5.0  8,082  6,619        0.0  0.0   
9   23,000  50,000  2,000  2,000      4.3  4.8  8,082  6,619        0.0  0.0   
10  23,000  50,000  2,000  2,000      4.4  4.9  8,082  6,619        0.0  0.0   
11  23,000  50,000  2,000  2,000      4.5  5.0  8,082  6,619        0.0  0.0   

   oprtg_rate          PGI           EGI        oprtg_cst         NOI         \
          nml   cld    nml    cld    nml    cld       nml  cld    nml    cld   
0        14.5  14.5  2,150  3,683  2,150  3,683       312  534  1,838  3,149   
1        14.5  14.5  2,150  3,683  2,150  3,683       312  534  1,838  3,149   
2        14.5  14.5  2,150  3,683  2,150  3,683       312  534  1,838  3,149   
3        14.5  14.5  2,248  3,843  2,248  3,843       326  557  1,922  3,286   
4        14.5  14.5  2,248  3,843  2,248  3,843       326  557  1,922  3,286   
5        14.5  14.5  2,248  3,843  2,248  3,843       326  557  1,922  3,286   
6        14.5  14.5  2,345  4,003  2,345  4,003       340  580  2,005  3,423   
7        14.5  14.5  2,345  4,003  2,345  4,003       340  580  2,005  3,423   
8        14.5  14.5  2,345  4,003  2,345  4,003       340  580  2,005  3,423   
9        14.5  14.5  2,443  4,163  2,443  4,163       354  604  2,089  3,560   
10       14.5  14.5  2,443  4,163  2,443  4,163       354  604  2,089  3,560   
11       14.5  14.5  2,443  4,163  2,443  4,163       354  604  2,089  3,560   

     value                   
       nml     cld      sum  
0   42,745  65,601  108,345  
1   41,773  64,262  106,035  
2   40,845  62,977  103,822  
3   44,689  68,454  113,143  
4   43,674  67,057  110,730  
5   42,703  65,716  108,419  
6   46,634  71,307  117,941  
7   45,574  69,852  115,426  
8   44,561  68,455  113,016  
9   48,578  74,160  122,738  
10  47,474  72,647  120,121  
11  46,419  71,194  117,613

In [68]:
valuation_smry = val_mtrx.printlst()[['rent', 'cap_rate', 'value']]
data_valuation['valuation_smry'] = valuation_smry
R(valuation_smry, rate=val_mtrx.ctgr_rate_mlti)

rent         cap_rate        value                 
       nml     cld      nml  cld     nml     cld      sum
0   20,000  44,000      4.3  4.8  42,745  65,601  108,345
1   20,000  44,000      4.4  4.9  41,773  64,262  106,035
2   20,000  44,000      4.5  5.0  40,845  62,977  103,822
3   21,000  46,000      4.3  4.8  44,689  68,454  113,143
4   21,000  46,000      4.4  4.9  43,674  67,057  110,730
5   21,000  46,000      4.5  5.0  42,703  65,716  108,419
6   22,000  48,000      4.3  4.8  46,634  71,307  117,941
7   22,000  48,000      4.4  4.9  45,574  69,852  115,426
8   22,000  48,000      4.5  5.0  44,561  68,455  113,016
9   23,000  50,000      4.3  4.8  48,578  74,160  122,738
10  23,000  50,000      4.4  4.9  47,474  72,647  120,121
11  23,000  50,000      4.5  5.0  46,419  71,194  117,613

In [69]:
with open('data/valuation.pickle', 'wb') as fw:
    pickle.dump(data_valuation, fw)